In [ ]:
# Importamos librerias a utilizar
import pandas as pd
import numpy as np
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Conexion a GCP - BigQuery
import pydata_google_auth
from google.cloud import bigquery
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]
credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)
client = bigquery.Client(project='gasco-analytics', credentials=credentials)

In [ ]:
query_gasconnect = """
SELECT A.*,
      CASE  WHEN A.n_canales = 2 THEN 'ON-OFF'
            WHEN A.n_canales = 1 AND tipo_pedido = 'ONLINE' THEN 'ONLINE'
            WHEN A.n_canales = 1 AND tipo_pedido = 'OFFLINE' THEN 'OFFLINE'
      END AS TIPO_CLIENTE,
      B.predict_estufa,
      B.predict_parrilla,
      B.predict_cocina,
      B.predict_calefont
FROM(
      SELECT EXTRACT(YEAR FROM createdat) AS YEAR,
             CAST(TELEFONO AS INT64) AS TELEFONO,
             SUM(IND_TA) AS PEDIDOS_TA,
             SUM(KILOS_TA) AS KILOS_TA,
             COUNT(DISTINCT createdAT) AS TOTAL_PEDIDOS,
             SUM(total_kilos_pedido) AS TOTAL_KILOS,
             COUNT(DISTINCT MONTH) AS MESES_COMPRA,
             MIN(EXTRACT(DATE FROM createdAt)) AS FECHA_PRIMER_PEDIDO,
             MAX(EXTRACT(DATE FROM createdAt)) AS FECHA_ULTIMO_PEDIDO,
             AVG(dias_compra) AS frec_compra_promedio, 
             stddev(dias_compra) AS desv_frec_compra,
             COUNT(DISTINCT TIPO_PEDIDO_BIEN) AS n_canales, 
             MAX(TIPO_PEDIDO_BIEN) AS tipo_pedido 
      FROM(
            SELECT *, 
                  DATE_DIFF(EXTRACT(DATE FROM createdat),LAG(EXTRACT(DATE FROM createdat)) OVER(PARTITION BY Telefono ORDER BY EXTRACT(DATE FROM createdat)), DAY) AS dias_compra 
            FROM(
                  SELECT *,
                        EXTRACT(YEAR FROM createdAt) AS YEAR,
                        EXTRACT(MONTH FROM createdAt) as MONTH,
                        CASE WHEN mes in (5,6,7,8,9) THEN 1 ELSE 0 END AS IND_TA,
                        CASE WHEN mes in (5,6,7,8,9) THEN total_kilos_pedido ELSE 0 END AS KILOS_TA,
                        CASE WHEN tipo_pedido_bien = 'OFFLINE' THEN customerNoIfOffline
                              WHEN tipo_pedido_bien = 'ONLINE' THEN mobile
                              ELSE mobile END AS Telefono,
                        CASE WHEN status = 'COMPLETED' THEN 'COMPLETADO' ELSE 'NO COMPLETADO' END AS ESTADO_PEDIDO, 
                  FROM `gasco-analytics.gasco.pedidos_gasconnect_todos_javier`
                  WHERE CAST(createdAt as DATE) >= '2019-01-01' --SELECCIONA LOS PEDIDOS DE UN AÑO MOVIL
                  AND isAnulled IS NULL     --PEDIDO NO ANULADO
                  --AND mobile IS NOT NULL    --TELEFONO NO NULO
                  AND TIPO_PEDIDO_BIEN IN ('OFFLINE', 'ONLINE')
            )
            WHERE ESTADO_PEDIDO = 'COMPLETADO'
      )
      GROUP BY EXTRACT(YEAR FROM createdat), TELEFONO
) AS A   

LEFT JOIN `gasco-analytics.gasco_modelos.20210218_prediccion_modelo_uso_gas` AS B  
ON(A.TELEFONO = B.telefono)
"""

**Actualización de los datos: 2021-04-20**

In [ ]:
read_data = False
if read_data:
    df_anual = pd.read_csv('datos\datos_exploratorio_fuga.csv',sep = ';', index_col = 0)
else:
    query_job = client.query(query_gasconnect)  # API request
    #rows = query_job.result()  # Waits for query to finish
    df_anual = query_job.to_dataframe()

    #guardamos como csv los datos
    df_anual.to_csv('datos\datos_exploratorio_fuga.csv',sep = ';')

df_anual.head(5)

In [ ]:
df_anual['FECHA_PRIMER_PEDIDO'] = pd.to_datetime(df_anual['FECHA_PRIMER_PEDIDO'])
df_anual['FECHA_ULTIMO_PEDIDO'] = pd.to_datetime(df_anual['FECHA_ULTIMO_PEDIDO'])

df_anual['porcentaje_pedidos_TA'] = df_anual['PEDIDOS_TA']/df_anual['TOTAL_PEDIDOS']
df_anual['recencia'] = (date(2021, 4, 7) - df_anual['FECHA_ULTIMO_PEDIDO'].dt.date).dt.days

df_anual.head(2)

In [ ]:
df_2019 = df_anual[df_anual.YEAR == 2019]
df_2020 = df_anual[df_anual.YEAR == 2020]
df_2021 = df_anual[df_anual.YEAR == 2021]

clientes_TA_2020 = df_2020[df_2020.porcentaje_pedidos_TA == 1]
clientes_TA_2019 = df_2019[df_2019.porcentaje_pedidos_TA == 1]

print('clientes que compran en TA en 2020:',len(clientes_TA_2020))
print('clientes que compran en TA en 2019:',len(clientes_TA_2019))

clientes_TA_total = clientes_TA_2020.merge(clientes_TA_2019,
                                           how = 'outer',
                                           on = 'TELEFONO',
                                           suffixes=('_2020', '_2019'),
                                           indicator = True)
clientes_TA_total.to_csv('clientes_TA_2019_2020.csv',sep = ';')

In [ ]:
query_demanda = """
SELECT YEAR, 
       MONTH, 
       COUNT(DISTINCT TELEFONO) AS CLIENTES

FROM(
    SELECT *, 
            DATE_DIFF(EXTRACT(DATE FROM createdat),LAG(EXTRACT(DATE FROM createdat)) OVER(PARTITION BY Telefono ORDER BY EXTRACT(DATE FROM createdat)), DAY) AS dias_compra 
    FROM(
            SELECT *,
                EXTRACT(YEAR FROM createdAt) AS YEAR,
                EXTRACT(MONTH FROM createdAt) as MONTH,
                CASE WHEN mes in (5,6,7,8,9) THEN 1 ELSE 0 END AS IND_TA,
                CASE WHEN mes in (5,6,7,8,9) THEN total_kilos_pedido ELSE 0 END AS KILOS_TA,
                CASE WHEN tipo_pedido_bien = 'OFFLINE' THEN customerNoIfOffline
                        WHEN tipo_pedido_bien = 'ONLINE' THEN mobile
                        ELSE mobile END AS Telefono,
                CASE WHEN status = 'COMPLETED' THEN 'COMPLETADO' ELSE 'NO COMPLETADO' END AS ESTADO_PEDIDO, 
            FROM `gasco-analytics.gasco.pedidos_gasconnect_todos_javier`
            WHERE CAST(createdAt as DATE) >= '2019-01-01' --SELECCIONA LOS PEDIDOS DE UN AÑO MOVIL
            AND isAnulled IS NULL     --PEDIDO NO ANULADO
            --AND mobile IS NOT NULL    --TELEFONO NO NULO
            AND TIPO_PEDIDO_BIEN IN ('OFFLINE', 'ONLINE')
    )
    WHERE ESTADO_PEDIDO = 'COMPLETADO'
) 
GROUP BY YEAR, MONTH
"""

query_job = client.query(query_demanda)  # API request
#rows = query_job.result()  # Waits for query to finish
df_demanda_mensual = query_job.to_dataframe()
df_demanda_mensual.to_csv('demanda_mensual.csv',sep = ';')

In [ ]:
df_demanda_mensual.head()

In [ ]:
df_demanda_mensual['FECHA'] = pd.to_datetime(df_demanda_mensual.YEAR*10000+df_demanda_mensual.MONTH*100+1,format='%Y%m%d')
df_demanda_mensual.head(2)

In [ ]:
#years = df_demanda_mensual.YEAR.unique()

#fig, axs = plt.subplots(len(years), figsize = (15,5), sharex = True, sharey = True)
#fig.suptitle('Clientes Mensuales', fontsize=16)

#for i in range(len(years)):
#    df = df_demanda_mensual[df_demanda_mensual.YEAR == years[i]]
#    sns.lineplot(ax = axs[i], data = df, x = 'MONTH', y = 'CLIENTES')

f, ax = plt.subplots(figsize = (15,5))
#sns.set(style="whitegrid")
sns.lineplot(data = df_demanda_mensual,
             x = 'MONTH', y = 'CLIENTES', hue = 'YEAR',palette = 'hls')

#### Juntar estos dos graficos en uno

In [ ]:
f, ax = plt.subplots(figsize = (15,5))
sns.lineplot(data = df_demanda_mensual,
             x = 'FECHA', y = 'CLIENTES')

### Etiquetas de los clientes por año y comparacion con recencia

In [ ]:
today = date.today()
print(today)

In [ ]:
df_2021

#### Dataframe labels Fuga

In [ ]:
df_fuga = pd.read_csv('cortes_frecuencia.csv', sep = ';', index_col = 0)

In [ ]:
choices_NB = ['inacto','activo bajo','activo alto']

conditions_NB = [
       (df_fuga['MESES_COMPRA'] <= 3), 
       ((df_fuga['MESES_COMPRA'] > 3) & (df_fuga['MESES_COMPRA'] <= 7)),
       (df_fuga['MESES_COMPRA'] > 7)]

df_fuga['estados_no_binarios'] = np.select(conditions_NB, choices_NB, default=np.nan)

df_fuga.head(2)

In [ ]:
print(len(df_fuga))
print(df_fuga.TELEFONO.nunique())

In [ ]:
df_fuga[['estados_no_binarios','TELEFONO']].groupby('estados_no_binarios').count().reset_index()

In [ ]:
df_inactivos = df_fuga[df_fuga.estados_no_binarios == 'inacto']
df_clientes_nuevos = df_inactivos[df_inactivos['fecha_primera_compra'] >= '2021-04-01']
print(len(df_clientes_nuevos))

df_clientes_nuevos.MESES_COMPRA.unique()

In [ ]:
df_AA = df_fuga[df_fuga.estados_no_binarios == 'activo alto']
df_AA_abril = df_AA[df_AA['FECHA_ULTIMO_PEDIDO'] >= '2021-04-01'] 
print(len(df_AA),len(df_AA_abril))


df_AB = df_fuga[df_fuga.estados_no_binarios == 'activo bajo']
df_AB_abril = df_AB[df_AB['FECHA_ULTIMO_PEDIDO'] >= '2021-04-01'] 
print(len(df_AB),len(df_AB_abril))

In [ ]:
clientes_TA = pd.read_csv('clientes_TA_2019_2020.csv',sep = ';', index_col = 0)
clientes_TA['FECHA_PRIMER_PEDIDO_2020']= pd.to_datetime(clientes_TA['FECHA_PRIMER_PEDIDO_2020'])

print(len(clientes_TA))

clientes_TA.head(2)

In [ ]:
clientes_TA_mayo_2020 = clientes_TA[(clientes_TA.FECHA_PRIMER_PEDIDO_2020 >= '2020-05-01') &
                                    (clientes_TA.FECHA_PRIMER_PEDIDO_2020 <= '2020-05-31')]
print(clientes_TA_mayo_2020.TELEFONO.nunique())

In [ ]:
#clientes_TA_mayo_2020.FECHA_PRIMER_PEDIDO_2020.max()

# Codigo de labels

**Tareas relacionadas a la extraccion de datos:**

1. Generar fechas para año movil (**O**)
2. Iterar, de manera retrograda, las fechas (generar historia pasada) (**O**)
3. Generar query parametrizada por fechas (**O**)
4. Leer query desde archivo .txt (**O**)
5. Agregar columnas que identiquen periodo (fecha inicio periodo / fecha termino periodo) (**O**)
6. Generar datos de tres periodos (periodos cerrados de lo que va 2021) (**O**)


**Tareas relacionadas a la generacion de etiquetas**
1. Aplicar logicas simples (por meses de compra) a los clientes. Esto es para cada periodo (**O**)
2. A los clientes "Activos" aplicar segunda logica de contraste frecuencia vs recencia (**O**)
3. Agregar tasa de permanencia (en meses) a cada dataframe por periodo (**O**)
4. Considerar inclusion de "Clientes Nuevos", aquellos que tienen su primera compra en el último mes del periodo (**O**)
5. Considerar inclusion de ventana de tiempo de adopcion de Clientes nuevos (tiempo de permanencia en la compañia) (**X**)
6. Consolidar todos los criterios (**X**)
7. Generar evolutivo (**X**)
8. Dejar codigo "inteligente" (**X**)
9. Agregar Exclusivos TA (**X**)

In [1]:
## Librerias
import pandas as pd
import numpy  as np

import datetime
from dateutil.relativedelta import relativedelta

import os

In [2]:
## Conexion a GCP - BigQuery
import pydata_google_auth
from google.cloud import bigquery
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]
credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)
client = bigquery.Client(project='gasco-analytics', credentials=credentials)

In [3]:
## Preparacion pre iteracion

# Parametros
n_periodos = 4 #Cantidad de periodos a considerar hacia atras
fecha_referencia = datetime.datetime.today().date()
print(fecha_referencia)

fecha_ter_AM = fecha_referencia + relativedelta(days = -fecha_referencia.day)

fecha_ini_AM = fecha_ter_AM + relativedelta(months = -11)
fecha_ini_AM = fecha_ini_AM.replace(day = 1) 
print(fecha_ini_AM, fecha_ter_AM)

######################################################################
# cond_file = os.path.exists('datos_clientes_fuga_final.csv')
# print(cond_file)

# df_clientes = pd.read_csv('datos_clientes_fuga_final.csv', sep = ';', index_col = 0,
#                           parse_dates = ['FECHA_INI_AM','FECHA_TER_AM'],
#                           date_parser = pd.to_datetime)

# periodos = df_clientes.FECHA_INI_AM
# max_per = periodos.max()

# comparacion = max_per.date() == fecha_ini_AM 
# print(max_per, fecha_ini_AM)
# print(comparacion)
# #max_per.date()

######################################################################
#condicion = False

#if condicion:
#    fecha_ini_iter = 1
#    fecha_ter_iter = 2
#else:
#    fecha_ter_AM = fecha_referencia + relativedelta(days = -fecha_referencia.day)
#    
#    fecha_ini_AM = fecha_ter_AM + relativedelta(months = -11)
#    fecha_ini_AM = fecha_ini_AM.replace(day = 1) 

#print(fecha_ini_AM, fecha_ter_AM)

with open('querys\query_fuga_parametrizada.txt') as f:
    query_2 = f.read()

df = []

2021-04-22
2020-04-01 2021-03-31


In [ ]:
#Ciclo iterativo

for i in range(n_periodos):
    
    fecha_ini_iter = fecha_ini_AM + relativedelta(months = -i)
    fecha_ter_iter = fecha_ter_AM + relativedelta(months = -i)
    print(fecha_ini_iter, fecha_ter_iter)
    
    job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('INI_DATE', 'DATE', fecha_ini_iter),
        bigquery.ScalarQueryParameter('TER_DATE', 'DATE', fecha_ter_iter),
        ]
    )
    
    query_job = client.query(query_2, job_config = job_config)  # API request
    df_iter = query_job.to_dataframe()
    
    #########################################################
    #       Reglas Basicas actividad
    #########################################################
    choices_NB = ['inacto','activo bajo','activo alto']

    conditions_NB = [
           (df_iter['MESES_COMPRA'] <= 3), 
           ((df_iter['MESES_COMPRA'] > 3) & (df_iter['MESES_COMPRA'] <= 7)),
           (df_iter['MESES_COMPRA'] > 7)]

    choices_B = ['inacto','activo']

    conditions_B = [
           (df_iter['MESES_COMPRA'] <= 5), 
           (df_iter['MESES_COMPRA'] > 5)]

    df_iter['estados_no_binarios'] = np.select(conditions_NB, choices_NB, default=np.nan)
    df_iter['estados_binarios'] = np.select(conditions_B, choices_B, default=np.nan)
    
    #########################################################
    #       Meses de permanencia clientes
    #########################################################   
    
    def get_diff_by_month(x):
        num_months = (fecha_ter_iter.year - x.year)*12 + (fecha_ter_iter.month - x.month)
        return(num_months)

    def str_to_date(x):
        date = datetime.datetime.strptime(x, '%Y-%m-%d')
        return(date)

    #df_clientes['fecha_primera_compra'] = df_fuga_2.FECHA_PRIMER_PEDIDO.apply(str_to_date)
    #df_clientes['meses_permanencia'] = df_fuga_2.fecha_primera_compra.apply(get_diff_by_month) + 1

    df_iter['meses_permanencia'] = df_iter.FECHA_PRIMER_PEDIDO.apply(get_diff_by_month) + 1
    df_iter['tasa_meses'] = df_iter['MESES_COMPRA']/df_iter['meses_permanencia']
    
    
    #########################################################
    #    Detección de Clientes Nuevos (Inactivo -> Nuevo)
    #########################################################   
    
    df_iter['estados_no_binarios'] = np.where(df_iter['meses_permanencia'] == 1, 'nuevo', 
                                              df_iter['estados_no_binarios'])

    df_iter['estados_binarios'] = np.where(df_iter['meses_permanencia'] == 1, 'nuevo', 
                                           df_iter['estados_binarios'])

    df.append(df_iter)
    
df_clientes = pd.concat(df)
df_clientes.to_csv('datos_clientes_fuga_final.csv', sep = ';')

In [ ]:
df_clientes = pd.read_csv('datos_clientes_fuga_final.csv', sep = ';', index_col = 0)
#Si se leen los datos, cambiar formato de FECHAS, que se leen como str a date.
df_clientes.head(2)

In [ ]:
#df_clientes['estados_no_binarios'] = np.where(df_clientes['meses_permanencia'] == 1, 'nuevo', 
#                                              df_clientes['estados_no_binarios'])

#df_clientes['estados_binarios'] = np.where(df_clientes['meses_permanencia'] == 1, 'nuevo', 
#                                            df_clientes['estados_binarios'])

In [ ]:
periodos = df_clientes.FECHA_INI_AM.unique()
columnas = ['MESES_COMPRA','frec_compra_promedio'] #,'desv_frec_compra']

for filtro in periodos:
    print(filtro)
    df_iter = df_clientes[df_clientes['FECHA_INI_AM'] == filtro]
    n1 = len(df_iter)
    
    df_frec_mes =  df_iter[columnas].groupby('MESES_COMPRA').agg([('Q1', lambda x: x.quantile(0.25)),
               ('Q2', lambda x: x.quantile(0.5)),
               ('Q3', lambda x: x.quantile(0.75)),
               ('Q90', lambda x: x.quantile(.90)),
               ('max', max),
                ('mean',np.mean)]
               ).reset_index()
    #Para los con solo un mes de compra, comparamos contra el maximo
    df_frec_mes[('frec_compra_promedio','Q90')][0] = df_frec_mes[('frec_compra_promedio','max')][0]
    
    #Clientes con una sola compra, presentan valor NA en frecuencia de compra.
    df_iter['frec_compra_promedio'] = df_iter['frec_compra_promedio'].fillna(0)
 
    df_iter = df_iter.merge(df_frec_mes, on = 'MESES_COMPRA')

    df_iter = df_iter[df_iter['frec_compra_promedio'] <= df_iter[('frec_compra_promedio', 'Q90')]]
    n2 = len(df_iter)
    
    #Prueba higenica para segurarnos de mantener % de clientes.
    print(100*n2/n1)

df_iter.head(2)

In [ ]:
df_frec_mes

In [ ]:
df_frec_mes_2 =  df_iter[columnas].groupby('MESES_COMPRA').agg([('Q1', lambda x: x.quantile(0.25)),
               ('Q2', lambda x: x.quantile(0.5)),
               ('Q3', lambda x: x.quantile(0.75)),
               ('Q90', lambda x: x.quantile(.90)),
               ('max', max),
                ('mean',np.mean)]
               ).reset_index()
df_frec_mes_2

In [ ]:
df_iter.columns

In [ ]:
df_iter['FECHA_TER_AM'] = pd.to_datetime(df_iter['FECHA_TER_AM'])
df_iter['FECHA_ULTIMO_PEDIDO'] = pd.to_datetime(df_iter['FECHA_ULTIMO_PEDIDO'])

In [ ]:
df_iter['recencia_periodo'] = (df_iter['FECHA_TER_AM'] - df_iter['FECHA_ULTIMO_PEDIDO']).apply(lambda x: x.days)

In [ ]:
df_iter.head()

In [ ]:
conditions_fuga = [
       (df_iter['recencia_periodo'] >  df_iter[('frec_compra_promedio','Q3')]), 
       (df_iter['recencia_periodo'] <= df_iter[('frec_compra_promedio','Q3')])]

choices_fuga = ['fugado','no fugado']

df_iter['estados_fuga'] = np.select(conditions_fuga, choices_fuga, default=np.nan)

In [ ]:
df_iter.head()

In [ ]:
choices_NB = ['activo alto','activo bajo','inactivo','nuevo']

conditions_NB = [
     (df_iter['MESES_COMPRA'] > 7), #Activo Alto
     ((df_iter['MESES_COMPRA'] > 3)  & (df_iter['MESES_COMPRA'] <= 7)), #Activo Bajo
     ((df_iter['MESES_COMPRA'] <= 3) & (df_iter['meses_permanencia']  > 1)), #Inactivo
     ((df_iter['MESES_COMPRA'] <= 3) & (df_iter['meses_permanencia']  = 1))
]

#######################################################################################
#      Necesita calculo de cuantiles de frec_compra_promedio
#######################################################################################

conditions_fuga = [
       (df_iter['recencia_periodo'] >  df_iter[('frec_compra_promedio','Q3')]), 
       (df_iter['recencia_periodo'] <= df_iter[('frec_compra_promedio','Q3')])]

choices_fuga = ['fugado','no fugado']

df_iter['estado_fuga'] = np.select(conditions_fuga, choices_fuga, default=np.nan)
df_iter['estado_fuga'] = np.where(df_iter['meses_permanencia'] == 1, 'nuevo', 
                                  df_iter['estado_fuga'])

In [4]:
def add_quantile_columns(df, columnas = ['MESES_COMPRA','frec_compra_promedio']):
    
    df_frec_mes =  df[columnas].groupby(columnas[0]).agg([('Q1', lambda x: x.quantile(0.25)),
               ('Q2', lambda x: x.quantile(0.5)),
               ('Q3', lambda x: x.quantile(0.75)),
               ('Q90', lambda x: x.quantile(.90)),
               ('max', max),
                ('mean',np.mean)]
               ).reset_index()
    
    #Para los con solo un mes de compra, comparamos contra el maximo
    df_frec_mes[(columnas[1],'Q90')][0] = df_frec_mes[(columnas[1],'max')][0]
    
    #Clientes con una sola compra, presentan valor NA en frecuencia de compra.
    df[columnas[1]] = df[columnas[1]].fillna(0)
 
    df = df.merge(df_frec_mes, on = columnas[0])
    
    return(df)

In [8]:
#Ciclo iterativo
for i in range(n_periodos):
    
    fecha_ini_iter = fecha_ini_AM + relativedelta(months = -i)
    fecha_ter_iter = fecha_ter_AM + relativedelta(months = -i)
    print(fecha_ini_iter, fecha_ter_iter)
    
    job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('INI_DATE', 'DATE', fecha_ini_iter),
        bigquery.ScalarQueryParameter('TER_DATE', 'DATE', fecha_ter_iter),
        ]
    )
    
    query_job = client.query(query_2, job_config = job_config)  # API request
    df_iter = query_job.to_dataframe()
        
    #########################################################
    #       Meses de permanencia clientes
    #########################################################   
    
    def get_diff_by_month(x):
        num_months = (fecha_ter_iter.year - x.year)*12 + (fecha_ter_iter.month - x.month)
        return(num_months)

    def str_to_date(x):
        date = datetime.datetime.strptime(x, '%Y-%m-%d')
        return(date)

    #df_clientes['fecha_primera_compra'] = df_fuga_2.FECHA_PRIMER_PEDIDO.apply(str_to_date)
    #df_clientes['meses_permanencia'] = df_fuga_2.fecha_primera_compra.apply(get_diff_by_month) + 1

    df_iter['recencia_periodo'] = (df_iter['FECHA_TER_AM'] - df_iter['FECHA_ULTIMO_PEDIDO']).apply(lambda x: x.days)
    df_iter['meses_permanencia'] = df_iter.FECHA_PRIMER_PEDIDO.apply(get_diff_by_month) + 1
    df_iter['tasa_meses'] = df_iter['MESES_COMPRA']/df_iter['meses_permanencia']
    
    df_iter = add_quantile_columns(df_iter)
    
    #########################################################
    #       Reglas Basicas actividad
    #########################################################
    
    choices_NB = ['activo alto','activo bajo','inactivo','nuevo']

    conditions_NB = [
         (df_iter['MESES_COMPRA'] > 7), #Activo Alto
         ((df_iter['MESES_COMPRA'] > 3)  & (df_iter['MESES_COMPRA'] <= 7)), #Activo Bajo
         ((df_iter['MESES_COMPRA'] <= 3) & (df_iter['meses_permanencia']  > 1)), #Inactivo
         ((df_iter['MESES_COMPRA'] <= 3) & (df_iter['meses_permanencia']  == 1))
    ]

    df_iter['estado_actividad'] = np.select(conditions_NB, choices_NB, default=np.nan)

    #######################################################################################
    #      Necesita calculo de cuantiles de frec_compra_promedio
    #######################################################################################

    
    conditions_fuga = [
           (df_iter['recencia_periodo'] >  df_iter[('frec_compra_promedio','Q3')]), 
           (df_iter['recencia_periodo'] <= df_iter[('frec_compra_promedio','Q3')])]

    choices_fuga = ['fugado','no fugado']

    df_iter['estado_fuga'] = np.select(conditions_fuga, choices_fuga, default=np.nan)
    df_iter['estado_fuga'] = np.where(df_iter['meses_permanencia'] == 1, 'nuevo', 
                                      df_iter['estado_fuga'])
    
    df.append(df_iter)
df_clientes = pd.concat(df)
df_clientes.to_csv('datos_clientes_fuga_final.csv', sep = ';')

2020-04-01 2021-03-31


<ipython-input-4-fd475927e970>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frec_mes[(columnas[1],'Q90')][0] = df_frec_mes[(columnas[1],'max')][0]


2020-03-01 2021-02-28
2020-02-01 2021-01-31
2020-01-01 2020-12-31


In [9]:
df_clientes.head()

,FECHA_INI_AM,FECHA_TER_AM,TELEFONO,PEDIDOS_TA,KILOS_TA,TOTAL_PEDIDOS,TOTAL_KILOS,MESES_COMPRA,FECHA_PRIMER_PEDIDO,FECHA_ULTIMO_PEDIDO,...,meses_permanencia,tasa_meses,"(frec_compra_promedio, Q1)","(frec_compra_promedio, Q2)","(frec_compra_promedio, Q3)","(frec_compra_promedio, Q90)","(frec_compra_promedio, max)","(frec_compra_promedio, mean)",estado_actividad,estado_fuga
0,2020-04-01,2021-03-31,931854016.0,17,202,28,323,12,2020-04-05,2021-03-25,...,12,1.0,12.5,15.809524,19.444444,23.070476,31.545455,16.027434,activo alto,no fugado
1,2020-04-01,2021-03-31,982141587.0,15,165,26,330,12,2020-04-13,2021-03-31,...,12,1.0,12.5,15.809524,19.444444,23.070476,31.545455,16.027434,activo alto,no fugado
2,2020-04-01,2021-03-31,992120122.0,18,231,37,462,12,2020-04-03,2021-03-29,...,12,1.0,12.5,15.809524,19.444444,23.070476,31.545455,16.027434,activo alto,no fugado
3,2020-04-01,2021-03-31,996123477.0,11,115,23,247,12,2020-04-04,2021-03-22,...,12,1.0,12.5,15.809524,19.444444,23.070476,31.545455,16.027434,activo alto,no fugado
4,2020-04-01,2021-03-31,229695232.0,20,253,39,550,12,2020-04-14,2021-03-25,...,12,1.0,12.5,15.809524,19.444444,23.070476,31.545455,16.027434,activo alto,no fugado
